<center>
<p style="text-align:center"><img alt="Ragas" src="https://github.com/explodinggradients/ragas/blob/main/docs/_static/imgs/logo.png?raw=true" width="400"><br><a href="https://docs.arize.com/phoenix/">Phoenix Docs</a> | <a href="https://github.com/explodinggradients/ragas">Ragas</a> | <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
</p>
    </center>

## 1. Introduction

Building a baseline for a RAG pipeline is not usually difficult, but enhancing it to make it suitable for production and ensuring the quality of your responses is almost always hard. Choosing the right tools and parameters for RAG can itself be challenging when there is an abundance of options available. This tutorial shares a robust workflow for making the right choices while building your RAG and ensuring its quality.

This article covers how to evaluate, visualize and analyze your RAG using a combination of open-source libraries.  We will be using:

- [Ragas](https://docs.ragas.io/en/stable/) for synthetic test data generation and evaluation
- Arize AI’s [Phoenix](https://docs.arize.com/phoenix) for tracing, visualization, and cluster analysis
- [LlamaIndex](https://docs.llamaindex.ai/en/stable/) for building RAG pipelines

For the purpose of this article, we’ll be using data from arXiv papers about prompt-engineering to build the RAG pipeline.

ℹ️ This notebook requires an OpenAI API key.

This notebook was created as supplemental material for the Arize AI paper reading on [LLM Alignment](https://arxiv.org/abs/2308.05374).

*   [Blog](https://arize.com/blog/trustworthy-llms-a-survey-and-guideline-for-evaluating-large-language-models-alignment/)
*  [ Full Recording](https://youtu.be/yKN1f4Gkjro?si=rwrETLwdZ-PxUm7g)



## 2. Install Dependencies and Import Libraries

Run the cell below to install Git LFS, which we use to download our dataset.

In [ ]:
!git lfs install

Git LFS initialized.


Install and import Python dependencies.

In [ ]:
!pip install "ragas==0.1.4" pypdf "arize-phoenix>=3.20.0" "openai>=1.0.0"  "llama-index>0.10.0" "llama-index-callbacks-arize-phoenix>=0.1.4" pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 20.6 MB/s eta 0:00:00
  Installing build dependenc

In [ ]:
import pandas as pd

# Display the complete contents of dataframe cells.
pd.set_option("display.max_colwidth", None)

## 3. Setup

Set your OpenAI API key if it is not already set as an environment variable.

In [ ]:
import os
from getpass import getpass

import openai

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

🔑 Enter your OpenAI API key: ··········


Launch Phoenix in the background and setup auto-instrumentation for llama-index and LangChain so that your OpenInference spans and traces are sent to and collected by Phoenix. [OpenInference](https://github.com/Arize-ai/openinference/tree/main/spec) is an open standard built atop OpenTelemetry that captures and stores LLM application executions. It is designed to be a category of telemetry data that is used to understand the execution of LLMs and the surrounding application context, such as retrieval from vector stores and the usage of external tools such as search engines or APIs.

In [ ]:
import phoenix as px
from llama_index.core import set_global_handler
from phoenix.trace.langchain import LangChainInstrumentor

session = px.launch_app()

# Setup instrumentation for both llama-index and LangChain (used by Ragas)
set_global_handler("arize_phoenix")
LangChainInstrumentor().instrument()

🌍 To view the Phoenix app in your browser, visit https://qw6owj74z24-496ff2e9c6d22116-6006-colab.googleusercontent.com/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


## 4. Generate Your Synthetic Test Dataset

Curating a golden test dataset for evaluation can be a long, tedious, and expensive process that is not pragmatic — especially when starting out or when data sources keep changing. This can be solved by synthetically generating high quality data points, which then can be verified by developers. This can reduce the time and effort in curating test data by 90%.

Run the cell below to download a dataset of prompt engineering papers in PDF format from arXiv and read these documents using LlamaIndex.

In [ ]:
!pip install arxiv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6049 sha256=bdec6ab7a658bdcc63a819bc34b4449c0fc39330b2b267aa007763cf079d9867
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [ ]:
# Step 2: Import the arxiv package and download a paper
import arxiv

# Define the arXiv ID of the paper you want to download
arxiv_id = '2308.05374'

# Search for the paper using the arXiv ID
search = arxiv.Search(id_list=[arxiv_id])

# Get the result
paper = next(search.results())

# Download the PDF of the paper
paper.download_pdf(filename=f"{paper.title}.pdf")

print(f"Downloaded: {paper.title}")


<ipython-input-7-61e3c365e349>:11: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  paper = next(search.results())


Downloaded: Trustworthy LLMs: a Survey and Guideline for Evaluating Large Language Models' Alignment


In [ ]:
from llama_index.core import SimpleDirectoryReader

dir_path = "./Papers" #you will need to comfirm you have the correct path for where you saved your paper
reader = SimpleDirectoryReader(dir_path, num_files_limit=2)
documents = reader.load_data()

In [ ]:
from phoenix.trace import using_project
from ragas.testset.evolutions import multi_context, reasoning, simple
from ragas.testset.generator import TestsetGenerator

TEST_SIZE = 25

# generator with openai models
generator = TestsetGenerator.with_openai()

# set question type distribution
distribution = {simple: 0.5, reasoning: 0.25, multi_context: 0.25}

# generate testset
with using_project("ragas-testset"):
    testset = generator.generate_with_llamaindex_docs(
        documents, test_size=TEST_SIZE, distributions=distribution
    )
test_df = testset.to_pandas()
test_df.head()

<ipython-input-9-ba7c5823ed16>:8: DeprecationWarning: The function with_openai was deprecated in 0.1.4, and will be removed in the 0.2.0 release. Use from_langchain instead.
  generator = TestsetGenerator.with_openai()


embedding nodes:   0%|          | 0/248 [00:00<?, ?it/s]

WARNI [ragas.testset.docstore] Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/25 [00:00<?, ?it/s]

question  \
0                             What challenges do LLMs face in understanding causality and performing causal reasoning tasks?   
1                                                 How can the evaluation data be converted into training data for alignment?   
2                                                                                   How robust are LLMs to typos in prompts?   
3  Why is it important for Language Model Models (LLMs) to express uncertainty and abstain from answering certain questions?   
4                                                  What are some challenges and alternatives in the LLM alignment algorithm?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

You are free to change the question type distribution according to your needs. Since we now have our test dataset ready, let’s move on and build a simple RAG pipeline using LlamaIndex.

## 5. Build Your RAG Application With LlamaIndex

LlamaIndex is an easy to use and flexible framework for building RAG applications. For the sake of simplicity, we use the default LLM (gpt-3.5-turbo) and embedding models (openai-ada-2).

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from phoenix.trace import using_project


def build_query_engine(documents):
    vector_index = VectorStoreIndex.from_documents(
        documents,
        embed_model=OpenAIEmbedding(),
    )
    query_engine = vector_index.as_query_engine(similarity_top_k=2)
    return query_engine


with using_project("indexing"):
    # By assigning a project name, the instrumentation will send all the embeddings to the indexing project
    query_engine = build_query_engine(documents)

If you check Phoenix, you should see embedding spans from when your corpus data was indexed. Export and save those embeddings into a dataframe for visualization later in the notebook.

In [ ]:
from phoenix.trace.dsl.helpers import SpanQuery

client = px.Client()
corpus_df = px.Client().query_spans(
    SpanQuery().explode(
        "embedding.embeddings",
        text="embedding.text",
        vector="embedding.vector",
    ),
    project_name="indexing",
)
corpus_df.head()

text  \
context.span_id  position                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
47f00c4c8c0616d9 0         page_label: 1\nfile_path: /content/Papers/Trustworthy LLMs: a Survey and Guideline for Evaluating Large Language Models' Alignment.pdf\n\nTRUSTWORTHY LLM S:ASURVEY AND GUIDELINE FOR\nEVALUATING LARGE LANGUAGE MODELS ’ ALIGNMENT\nYang Liu∗Yuanshun Yao∗Jean-Francois Ton Xiaoying Zhang Ruocheng Guo\nHao Cheng Yegor Klochkov Muhammad Faaiz Taufiq Hang Li\nByteDance Research\nAugust 9, 2023\nABSTRACT\nEnsuring alignment, which refers to making models behave in accordance with human intentions [ 1,2],\nhas become a critical task before deploying large language models (LLMs) in real-world applications.\nFor instance, OpenAI devoted six months to iteratively aligning GPT-4 before its release [ 3]. However,\na major challenge faced by practitioners is the lack of clear guidance on evaluating whether LLM\noutputs align with social norms, values, and regulations. This obstacle hinders systematic iteration\nand deployment of LLMs. To address this issue, this paper presents a comprehensive survey of\nkey dimensions that are crucial to consider when assessing LLM trustworthiness. The survey\ncovers seven major categories of LLM trustworthiness: reliability, safety, fairness, resistance to\nmisuse, explainability and reasoning, adherence to social norms, and robustness. Each major\ncategory is further divided into several sub-categories, resulting in a tota

## 6. Evaluate Your LLM Application

Ragas provides a comprehensive list of metrics that can be used to evaluate RAG pipelines both component-wise and end-to-end.

To use Ragas, we first form an evaluation dataset comprised of a question, generated answer, retrieved context, and ground-truth answer (the actual expected answer for the given question).

In [ ]:
import pandas as pd
from datasets import Dataset
from phoenix.trace import using_project
from tqdm.auto import tqdm


def generate_response(query_engine, question):
    response = query_engine.query(question)
    return {
        "answer": response.response,
        "contexts": [c.node.get_content() for c in response.source_nodes],
    }


def generate_ragas_dataset(query_engine, test_df):
    test_questions = test_df["question"].values
    responses = [generate_response(query_engine, q) for q in tqdm(test_questions)]

    dataset_dict = {
        "question": test_questions,
        "answer": [response["answer"] for response in responses],
        "contexts": [response["contexts"] for response in responses],
        "ground_truth": test_df["ground_truth"].values.tolist(),
    }
    ds = Dataset.from_dict(dataset_dict)
    return ds


with using_project("llama-index"):
    ragas_eval_dataset = generate_ragas_dataset(query_engine, test_df)

ragas_evals_df = pd.DataFrame(ragas_eval_dataset)
ragas_evals_df.head()

  0%|          | 0/25 [00:00<?, ?it/s]

question  \
0                             What challenges do LLMs face in understanding causality and performing causal reasoning tasks?   
1                                                 How can the evaluation data be converted into training data for alignment?   
2                                                                                   How robust are LLMs to typos in prompts?   
3  Why is it important for Language Model Models (LLMs) to express uncertainty and abstain from answering certain questions?   
4                                                  What are some challenges and alternatives in the LLM alignment algorithm?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       answer  \
0                                                                                                                                                                                                                                                                                      LLMs face challenges in understanding causality and performing causal reasoning tasks, particularly in inferring the sufficient causes of events. This difficulty arises because inferring the sufficient causes requires considering a large set of counterfactual questions. While LLMs can be accurate in making inferences of necessary cause, their accuracy in inferring sufficient cause is notably lower due to the complexity of evaluating various counterfactual scenarios.   
1                                                                                                                                                                                                                                                                                                                                                                                                                         The evaluation data can be converted into training data for alignment by leveraging the existing high-quality LLMs to judge if a model passes a certain test or not. This process can accelerate the evaluation task from manual work to a more automated approach, reducing the reliance on human labelers and speeding up the evaluation process.   
2                                                                                                                                                                                                                                                                                            LLMs show varying levels of robustness to typos in prompts. The evaluation results indicate that none of the LLMs are extremely robust to typo attacks. Among the well-aligned LLMs, davinci has the smallest drop in consistency, while flan-t5-xxl exhibits the least amount of consistency downgrade. On the other hand, ChatGPT and GPT-4 demonstrate vulnerability against typo attacks, often providing inconsistent answers or marking the typoed prompts as nonsensical.   
3                                                                                                                                                                                                                   Expressing uncertainty and abstaining from answering certain questions is important for Language Models (LLMs) to maintain 

Check out Phoenix to view your LlamaIndex application traces.